In [ ]:
import pandas as pd

new_inclinometria = pd.DataFrame(
    {
        'Name' : [],
        'MD' : [],
        'INCL' : [],
        'AZIM' : [],
        'ELONG' : [],
        'DISPL' : [],
        'Z' : [],
        'TVD' : [],
        'DY' : [],
        'DX' : [],
        'X' : [],
        'Y' : [],
        'Магнитное склонение' : []
    }
)

files_names = os.listdir('Inclinometria')
print(f'Файлов в папке: {len(files_names)}')

for name in files_names:
    if name [-10:-5] == 'пилот' or name[-9:-4] == 'пилот':
        well_name = name[:4]
    else:
        well_name = name[:4] + 'Г'

    incl_head = pd.read_excel(f'Inclinometria/{name}', sheet_name = '2. Инклинометрия')

    magnetic = float(incl_head['Unnamed: 4'].loc[3]) * -1

    x = round(float(incl_head['Unnamed: 10'].loc[9]), 3)
    y = round(float(incl_head['Unnamed: 10'].loc[8]), 3)

    if y < x:
        xm = y
        ym = x
    else:
        xm = x
        ym = y
    
    print(f'{well_name}: X → {round(xm + 21981, 3)}, Y → {round(ym + 4500043, 3)}, Магнитное склонение → {magnetic}')

    incl = pd.read_excel(f'Inclinometria/{name}', sheet_name = '2. Инклинометрия', skiprows = 23)
    icnl = incl.drop([incl.columns[2], incl.columns[6], incl.columns[11]], axis = 1)
    incl = incl.dropna(how = 'any', thresh = 8, axis = 0)

    incl = incl.rename(columns = {
        incl.columns[0] : 'MD',
        incl.columns[1] : 'INCL',
        incl.columns[2] : 'AZIM',
        incl.columns[3] : 'ELONG',
        incl.columns[4] : 'DISPL',
        incl.columns[5] : 'Z',
        incl.columns[6] : 'TVD',
        incl.columns[7] : 'DY',
        icnl.columns[8] : 'DX'
    })
    incl = incl.tail(-2)

    incl['Name'] = well_name
    incl['MD'] = incl['MD'].astype('str')
    incl['MD'] = incl['MD'].apply(lambda x: x[:-2] if x[-2:] == '**' else x)
    icnl['MD'] = incl['MD'].apply(lambda x: x[:-1] if x[-1:] == '*' else x)
    incl['X'] = round(xm + 21981, 3)
    incl['Y'] = round(ym + 4500043, 3)
    incl['Магнитное склонение'] = magnetic

    incl['X'] = incl['X'].drop_duplicates(keep = 'first')
    incl['Y'] = incl['Y'].drop_duplicates(keep = 'first')
    incl['Магнитное склонение'] = incl['Магнитное склонение'].drop_duplicates(keep = 'first')

    incl.reset_index(drop = True, inplace = True)

    values = {'AZIM' : -999.25}
    incl = incl.fillna(values)

    for col in incl.columns:
        if col != 'Name':
            incl[col] = incl[col].astype('float')

    incl = pd.DataFrame(incl)
    new_inclinometria = pd.concat([new_inclinometria, incl])

new_inclinometria.to_excel('New/New_inclinometria.xlsx', index = False)
print('Программа успешно завершена')